### (간략) 구문 분석

1. spacy 구문분석

In [1]:
!pip install spacy
!python -m spacy download ko_core_news_sm

Defaulting to user installation because normal site-packages is not writeable
     ---------------------------------------- 0.0/107.3 kB ? eta -:--:--
     -------------------------------------- 107.3/107.3 kB 6.1 MB/s eta 0:00:00
   ---------------------------------------- 0.0/11.7 MB ? eta -:--:--
   ---------------- ----------------------- 4.9/11.7 MB 157.6 MB/s eta 0:00:01
   ------------------------- -------------- 7.5/11.7 MB 96.4 MB/s eta 0:00:01
   --------------------------- ------------ 7.9/11.7 MB 63.1 MB/s eta 0:00:01
   ------------------------------------ --- 10.7/11.7 MB 59.5 MB/s eta 0:00:01
   ---------------------------------------- 11.7/11.7 MB 46.9 MB/s eta 0:00:00
   ---------------------------------------- 0.0/182.0 kB ? eta -:--:--
   --------------------------------------- 182.0/182.0 kB 10.7 MB/s eta 0:00:00
   ---------------------------------------- 0.0/122.4 kB ? eta -:--:--
   ---------------------------------------- 122.4/122.4 kB ? eta 0:00:00
   --------

In [2]:
import spacy

In [3]:
# 한국어 모델 로드 # "ko_core_news_sm"
nlp = spacy.load("ko_core_news_sm")

In [4]:
# 구문트리
doc = nlp("이것은 한국어로 된 문장입니다.")

In [5]:
# 구문트리 출력하기 
# dep : 관계

for token in doc:
    print(f"token : {token.text} || token.dep_: {token.dep_} || token.head.text : {token.head.text}")

token : 이것은 || token.dep_: dislocated || token.head.text : 문장입니다
token : 한국어로 || token.dep_: nsubj || token.head.text : 된
token : 된 || token.dep_: ccomp || token.head.text : 문장입니다
token : 문장입니다 || token.dep_: ROOT || token.head.text : 문장입니다
token : . || token.dep_: punct || token.head.text : 문장입니다


In [8]:
# displacy 시각화
from spacy import displacy

displacy.render(doc, style='dep', jupyter=True)

In [9]:
import pandas as pd

In [21]:
# 데이터 가져와서 구문 분석

train_df = pd.read_csv('https://raw.githubusercontent.com/e9t/nsmc/master/ratings_train.txt', sep='\t', encoding='utf-8')
sample_data = train_df['document'][100]
sample_data

'신카이 마코토의 작화와,미유와 하나카나가 연기를 잘해줘서 더대박이였다.'

In [23]:
doc2 = nlp(sample_data)

doc2

신카이 마코토의 작화와,미유와 하나카나가 연기를 잘해줘서 더대박이였다.

In [24]:
# 출력

for token in doc2:
    print(f"token : {token.text} || token.dep_: {token.dep_} || token.head.text : {token.head.text}")

token : 신카이 || token.dep_: amod || token.head.text : 마코토의
token : 마코토의 || token.dep_: dislocated || token.head.text : 더대박이였다
token : 작화와 || token.dep_: nmod || token.head.text : 연기를
token : , || token.dep_: punct || token.head.text : 작화와
token : 미유와 || token.dep_: conj || token.head.text : 작화와
token : 하나카나가 || token.dep_: conj || token.head.text : 작화와
token : 연기를 || token.dep_: obj || token.head.text : 잘해줘서
token : 잘해줘서 || token.dep_: ROOT || token.head.text : 잘해줘서
token : 더대박이였다 || token.dep_: conj || token.head.text : 잘해줘서
token : . || token.dep_: punct || token.head.text : 더대박이였다


In [26]:
# displacy 시각화
from spacy import displacy
displacy.render(doc2, style='dep', jupyter=True)

2. NLTK 구문분석

In [18]:
import konlpy
import nltk
from ckonlpy.tag import Twitter

In [19]:
# 형태소 분석기
okt = Twitter()

C:\Users\BIG3-06\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\konlpy\tag\_okt.py:17: UserWarning: "Twitter" has changed to "Okt" since KoNLPy v0.4.5.
  warn('"Twitter" has changed to "Okt" since KoNLPy v0.4.5.')


In [20]:
# 사용자 사전 추가

okt.add_dictionary("미유와", "Noun")
okt.add_dictionary("하나카나", "Noun")

In [22]:
# twitter 형태소 분석

words = okt.pos(sample_data)
words

[('신카이', 'Noun'),
 ('마코토', 'Noun'),
 ('의', 'Josa'),
 ('작화', 'Noun'),
 ('와', 'Josa'),
 (',', 'Punctuation'),
 ('미유와', 'Noun'),
 ('하나카나', 'Noun'),
 ('가', 'Josa'),
 ('연기', 'Noun'),
 ('를', 'Josa'),
 ('잘', 'VerbPrefix'),
 ('해줘서', 'Verb'),
 ('더', 'Noun'),
 ('대박', 'Noun'),
 ('이', 'Josa'),
 ('였다', 'Verb'),
 ('.', 'Punctuation')]

- <N.*>: 하나 이상의 명사 (N으로 시작하는 품사 태그)  
*: 0번 이상의 앞의 패턴에 일치하는 항목  
<Suffix>?: 옵션으로 하나의 접미사 (Suffix로 끝나는 품사 태그)

- <V.*>: 하나 이상의 명사 (V으로 시작하는 품사 태그)

- <A.*>: 하나 이상의 명사 (A으로 시작하는 품사 태그)

In [29]:
# 문법을 지정해서 '구'형태로 묶기
grammar = """
NP: {<N.*>*<Suffix>?}  # N 뒤에 임의의 문자열 나타남. N번 반복, 
VP: {<V.*>*}
AP: {<A.*>*}
"""
parser = nltk.RegexpParser(grammar=grammar)
chunks = parser.parse(words)
print(chunks.pprint()) # pprint() 함수는 데이터를 보기 좋게 출력할 수 있는 텍스트 형식으로 변환

(S
  (NP 신카이/Noun 마코토/Noun)
  의/Josa
  (NP 작화/Noun)
  와/Josa
  ,/Punctuation
  (NP 미유와/Noun 하나카나/Noun)
  가/Josa
  (NP 연기/Noun)
  를/Josa
  (VP 잘/VerbPrefix 해줘서/Verb)
  (NP 더/Noun 대박/Noun)
  이/Josa
  (VP 였다/Verb)
  ./Punctuation)
None


In [31]:
# 시각화

chunks.draw() # 새 창에 이미지 보여짐.